# **Using BERT pretrained transformer model for text classification** 

Following guide from https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b


In [ ]:
# Mount google drive to access google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CS3244_Project/
source_folder = '/content/drive/MyDrive/CS3244_Project/'
destination_folder = '/content/drive/My Drive/CS3244Project/BERT_MODEL' # To save model and its metrics

/content/drive/.shortcut-targets-by-id/1HIneZcYa6myY-nOkYO6PcrQ9axBSlL2R/CS3244_Project


# **Don't need to do any of this**

In [ ]:
ls

 BERT.ipynb      Old/                      sarcasm_valid.csv
 BERT_MODEL/     Sarcasm_dataprep.ipynb    SVM.ipynb
'CS3244 LR'      sarcasm_test.csv          test-balanced.csv
 LSTM.ipynb      sarcasm_train.csv         train-balanced-sarcasm.csv
 lstm_model.pt   sarcasm_train_valid.csv   Untitled0.ipynb


In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('sarcasm_train.csv',engine='python')
test = pd.read_csv('sarcasm_test.csv',engine='python')
train.head(5)

,label,comment,score,ups,downs,parent_comment,word_count,capitilised_frequency,punctuation_frequency,comment_processed,parent_comment_processed
0,0,*eventually.,18,18,0,WE ARE ALL GOING TO DIE!,1,0.000000,0.166667,eventually,we are all going to die
1,1,The average American.,-1,-1,0,"Non-Americans of reddit, What are some example...",3,0.095238,0.047619,the average american,nonamericans of reddit what are some examples ...
2,1,Not sure if I can handle all of that edge in t...,15,15,0,GIF dump,15,0.041667,0.013889,not sure if i can handle all of that edge in t...,gif dump
3,0,Crown Vic?,13,13,0,"Best car for a moron. To be honest, I don't th...",2,0.200000,0.100000,crown vic,best car for a moron to be honest i dont think...
4,1,"No, humans are clearly above using military po...",3,3,0,So there's a treaty that no country are able t...,16,0.010309,0.010309,no humans are clearly above using military pow...,so theres a treaty that no country are able to...


In [ ]:
print(train.shape)

(808617, 13)


In [ ]:
train.drop(['score', 'ups', 'downs', 'parent_comment', 'word_count', 'capitilised_frequency', 'punctuation_frequency', 'subreddit_code', 'author_code', 'comment_processed', 'parent_comment_processed'], axis=1, inplace=True)

KeyError: ignored

In [ ]:
test.drop(['score', 'ups', 'downs', 'parent_comment', 'word_count', 'capitilised_frequency', 'punctuation_frequency', 'subreddit_code', 'author_code', 'comment_processed', 'parent_comment_processed'], axis=1, inplace=True)

In [ ]:
train.head()
test.head()


,label,comment
0,1,Well in that case it's much easier to post a c...
1,1,"Absolutely not, everyone must conform to my op..."
2,0,Also pulled totw Reinhart but meh
3,1,Just like Dead Space was rubbish?
4,0,"Or occasionally their kids, according to some ..."


# **Install and import libraries**

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 3.3 MB 37.6 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries
#import torchtext.data
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [ ]:
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  print(device)

cuda:0


# **Prepare dataset**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Model parameter
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('comment', text_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='sarcasm_train.csv', validation='sarcasm_valid.csv',
                                           test='sarcasm_test.csv', format='CSV', fields=fields, skip_header=True)




Token indices sequence length is longer than the specified maximum sequence length for this model (2002 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Iterators
train_iter = BucketIterator(train, batch_size=64, sort_key=lambda x: len(x.comment),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=64, sort_key=lambda x: len(x.comment),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)

https://gmihaila.medium.com/better-batches-with-pytorchtext-bucketiterator-12804a545e2a

# **Build model**

In [ ]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [ ]:
# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [ ]:
print(len(train_iter))

14214


# **Training Function**

In [ ]:
# Training Function

def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 5,
          eval_every = len(train_iter) // 4,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []
    count = 0
    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (labels, comment), _ in train_loader:
            labels = labels.type(torch.LongTensor)           
            labels = labels.to(device)
            comment = comment.type(torch.LongTensor)  
            comment = comment.to(device)
            output = model(comment, labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1
            print(global_step)

            # evaluation step
            if global_step % eval_every == 0:
                print("here")
                model.eval()
                with torch.no_grad():                    

                    # validation loop
                    for (labels, comment), _ in valid_loader:
                        labels = labels.type(torch.LongTensor)           
                        labels = labels.to(device)
                        comment = comment.type(torch.LongTensor)  
                        comment = comment.to(device)
                        output = model(comment, labels)
                        loss, _ = output
                        
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

NameError: ignored

Training Function without **validation**

In [ ]:
# Training Function

def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 1,
          eval_every = len(train_iter) // 4,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []
    count = 0
    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (labels, comment), _ in train_loader:
            labels = labels.type(torch.LongTensor)           
            labels = labels.to(device)
            comment = comment.type(torch.LongTensor)  
            comment = comment.to(device)
            output = model(comment, labels)
            print(output)

            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % 100 == 0:
                print("here")
                model.eval()               

                # evaluation
                average_train_loss = running_loss / eval_every
                train_loss_list.append(average_train_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss))
    save_checkpoint(file_path + '/' + 'model.pt', model)            
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, global_steps_list)
    print('Finished Training!')

# **Will probably take a few days to train this lol..**

In [ ]:
model = BERT().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

train(model=model, optimizer=optimizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored

With validation

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 

Without Validation

In [ ]:
train_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 

# **Evaluation**

In [ ]:
# Evaluation Function

def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for (labels, comment), _ in test_loader:

                labels = labels.type(torch.LongTensor)           
                labels = labels.to(device)
                comment = comment.type(torch.LongTensor)  
                comment = comment.to(device)
                output = model(comment, labels)

                _, output = output
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())
    
    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=[1,0], digits=4))
    
    cm = confusion_matrix(y_true, y_pred, labels=[1,0])
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')

    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')

    ax.xaxis.set_ticklabels(['FAKE', 'REAL'])
    ax.yaxis.set_ticklabels(['FAKE', 'REAL'])

In [ ]:
best_model = BERT().to(device)

load_checkpoint(destination_folder + '/model.pt', best_model)

evaluate(best_model, test_iter)